In [1]:
from random import random, choice

In [40]:
def evaluate_condition(observation, condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if 'NOT ' + feature in condition and observation[feature] == True:
            truth_value_of_condition = False
            break
    return truth_value_of_condition

In [41]:
class Memory:
    def __init__(self, forget_value, memorize_value, memory):
        self.memory = memory
        self.forget_value = forget_value
        self.memorize_value = memorize_value
    
    def get_memory(self):
        return self.memory
    
    def get_literals(self):
        return list(self.memory.keys())
    
    def get_condition(self):
        condition = []
        for literal in self.memory:
            if self.memory[literal] >= 6:
                condition.append(literal)
        return condition
        
    def memorize(self, literal):
        if random() <= self.memorize_value and self.memory[literal] < 10:
            self.memory[literal] += 1
            
    def forget(self, literal):
        if random() <= self.forget_value and self.memory[literal] > 1:
            self.memory[literal] -= 1
            
    def memorize_always(self, literal):
        if  self.memory[literal] < 10:
            self.memory[literal] += 1

In [42]:
def classify(observation, recurrence_rules, non_recurrence_rules):
    vote_sum = 0
    for recurrence_rule in recurrence_rules:
        if evaluate_condition(observation, recurrence_rule.get_condition()) == True:
            vote_sum += 1
    for non_recurrence_rule in non_recurrence_rules:
        if evaluate_condition(observation, non_recurrence_rule.get_condition()) == True:
            vote_sum -= 1
    if vote_sum >= 0:
        return "Recurrence"
    else:
        return "Non Recurrence"

In [43]:
def type_i_feedback(observation, memory):
    remaining_literals = memory.get_literals()
    if evaluate_condition(observation, memory.get_condition()) == True:
        for feature in observation:
            if observation[feature] == True:
                memory.memorize(feature)
                remaining_literals.remove(feature)
            elif observation[feature] == False:
                memory.memorize('NOT ' + feature)
                remaining_literals.remove('NOT ' + feature)
    for literal in remaining_literals:
        memory.forget(literal)

def type_ii_feedback(observation, memory):
    if evaluate_condition(observation, memory.get_condition()) == True:
        for feature in observation:
            if observation[feature] == False:
                memory.memorize_always(feature)
            elif observation[feature] == True:
                memory.memorize_always('NOT ' + feature)

## First part of assignment (Step 2 - Step 4)
Manually adding the dataset, the rules and classify the patients with the three rules

In [28]:
patients_recurrence = [
    {'menopause-lt40':False, 'menopause-ge40':True, 'menopause-premeno':False, 'inv-nodes-0-2':False, 'inv-nodes-3-5':True, 'inv-nodes-6-8':False, 'deg-malign-1': False, 'deg-malign-2': False, 'deg-malign-3':True},
    {'menopause-lt40':False, 'menopause-ge40':True, 'menopause-premeno':False, 'inv-nodes-0-2':False, 'inv-nodes-3-5':False, 'inv-nodes-6-8':True, 'deg-malign-1': False, 'deg-malign-2': False, 'deg-malign-3':True},
    {'menopause-lt40':False, 'menopause-ge40':False, 'menopause-premeno':True, 'inv-nodes-0-2':True, 'inv-nodes-3-5':False, 'inv-nodes-6-8':False, 'deg-malign-1': False, 'deg-malign-2': False, 'deg-malign-3':True},
]

patients_non_recurrence = [
    {'menopause-lt40':True, 'menopause-ge40':False, 'menopause-premeno':False, 'inv-nodes-0-2':True, 'inv-nodes-3-5':False, 'inv-nodes-6-8':False, 'deg-malign-1': False, 'deg-malign-2': False, 'deg-malign-3':True},
    {'menopause-lt40':False, 'menopause-ge40':True, 'menopause-premeno':False, 'inv-nodes-0-2':True, 'inv-nodes-3-5':False, 'inv-nodes-6-8':False, 'deg-malign-1': False, 'deg-malign-2': True, 'deg-malign-3':False},
    {'menopause-lt40':False, 'menopause-ge40':False, 'menopause-premeno':True, 'inv-nodes-0-2':True, 'inv-nodes-3-5':False, 'inv-nodes-6-8':False, 'deg-malign-1': True, 'deg-malign-2': False, 'deg-malign-3':False},
]

In [44]:
#Defining the rules manually
predefined_rule_1 = Memory(0.0, 1.0,{'deg-malign-3':10,'NOT menopause-lt40':10})
predefined_rule_2 = Memory(0.0, 1.0,{'deg-malign-3':10,'NOT menopause-lt40':10})
predefined_rule_3 = Memory(0.0, 1.0,{'inv-nodes-0-2':10})

#Combining recurrence rules togther
predefined_recurrence_rules = [predefined_rule_1, predefined_rule_2]
predefined_non_recurrence_rules = [predefined_rule_3]

#Printing conditions
print("IF " + " AND ".join(predefined_rule_1.get_condition()) + " THEN Recurrence")
print("IF " + " AND ".join(predefined_rule_2.get_condition()) + " THEN Recurrence")
print("IF " + " AND ".join(predefined_rule_3.get_condition()) + " THEN Non-Recurrence")


#Classifying patients
print(classify(patients_recurrence[0], predefined_recurrence_rules, predefined_non_recurrence_rules))
print(classify(patients_recurrence[1], predefined_recurrence_rules, predefined_non_recurrence_rules))
print(classify(patients_recurrence[2], predefined_recurrence_rules, predefined_non_recurrence_rules))

print(classify(patients_non_recurrence[0], predefined_recurrence_rules, predefined_non_recurrence_rules))
print(classify(patients_non_recurrence[1], predefined_recurrence_rules, predefined_non_recurrence_rules))
print(classify(patients_non_recurrence[2], predefined_recurrence_rules, predefined_non_recurrence_rules))

IF deg-malign-3 AND NOT menopause-lt40 THEN Recurrence
IF deg-malign-3 AND NOT menopause-lt40 THEN Recurrence
IF inv-nodes-0-2 THEN Non-Recurrence
Recurrence
Recurrence
Recurrence
Non Recurrence
Non Recurrence
Non Recurrence


## Step 5 and Step 6
Learning new rules with forget value 0.8 and memorize value 0.2

In [58]:
recurrence_rule_1 = Memory(0.8, 0.2,
                           {'menopause-lt40':5,
                            'NOT menopause-lt40':5,
                            'menopause-ge40':5,
                            'NOT menopause-ge40':5,
                            'menopause-premeno':5,
                            'NOT menopause-premeno':5,
                            'inv-nodes-0-2':5,
                            'NOT inv-nodes-0-2':5,
                            'inv-nodes-3-5':5,
                            'NOT inv-nodes-3-5':5,
                            'inv-nodes-6-8':5,
                            'NOT inv-nodes-6-8':5,
                            'deg-malign-1':5,
                            'NOT deg-malign-1':5,
                            'deg-malign-2':5,
                            'NOT deg-malign-2':5,
                            'deg-malign-3':5,
                            'NOT deg-malign-3':5
                           })

non_recurrence_rule_1 = Memory(0.8, 0.2,
                           {'menopause-lt40':5,
                            'NOT menopause-lt40':5,
                            'menopause-ge40':5,
                            'NOT menopause-ge40':5,
                            'menopause-premeno':5,
                            'NOT menopause-premeno':5,
                            'inv-nodes-0-2':5,
                            'NOT inv-nodes-0-2':5,
                            'inv-nodes-3-5':5,
                            'NOT inv-nodes-3-5':5,
                            'inv-nodes-6-8':5,
                            'NOT inv-nodes-6-8':5,
                            'deg-malign-1':5,
                            'NOT deg-malign-1':5,
                            'deg-malign-2':5,
                            'NOT deg-malign-2':5,
                            'deg-malign-3':5,
                            'NOT deg-malign-3':5
                           })

In [59]:
# Feedback for recurrence rule
for i in range(500):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 0:
        type_i_feedback(patients_recurrence[observation_id], recurrence_rule_1)
    else:
        type_ii_feedback(patients_non_recurrence[observation_id], recurrence_rule_1)

# Feedback for non recurrence rule
for i in range(500):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 0:
        type_i_feedback(patients_non_recurrence[observation_id], non_recurrence_rule_1)
    else:
        type_ii_feedback(patients_recurrence[observation_id], non_recurrence_rule_1)


In [60]:
print(classify(patients_recurrence[0], [recurrence_rule_1], [non_recurrence_rule_1]))
print(classify(patients_recurrence[1], [recurrence_rule_1], [non_recurrence_rule_1]))
print(classify(patients_recurrence[2], [recurrence_rule_1], [non_recurrence_rule_1]))

print(classify(patients_non_recurrence[0], [recurrence_rule_1], [non_recurrence_rule_1]))
print(classify(patients_non_recurrence[1], [recurrence_rule_1], [non_recurrence_rule_1]))
print(classify(patients_non_recurrence[2], [recurrence_rule_1], [non_recurrence_rule_1]))

print("\nIF " + " AND ".join(recurrence_rule_1.get_condition()) + " THEN Recurrence \n")

print("IF " + " AND ".join(non_recurrence_rule_1.get_condition()) + " THEN Non-Recurrence")

Recurrence
Recurrence
Recurrence
Recurrence
Non Recurrence
Non Recurrence

IF NOT menopause-lt40 AND NOT deg-malign-1 AND NOT deg-malign-2 AND deg-malign-3 THEN Recurrence 

IF inv-nodes-0-2 AND NOT deg-malign-3 THEN Non-Recurrence


## Step 7
Learning rules with forget value 0.5 and memorize value 0.5

In [61]:
recurrence_rule_2 = Memory(0.5, 0.5,
                           {'menopause-lt40':5,
                            'NOT menopause-lt40':5,
                            'menopause-ge40':5,
                            'NOT menopause-ge40':5,
                            'menopause-premeno':5,
                            'NOT menopause-premeno':5,
                            'inv-nodes-0-2':5,
                            'NOT inv-nodes-0-2':5,
                            'inv-nodes-3-5':5,
                            'NOT inv-nodes-3-5':5,
                            'inv-nodes-6-8':5,
                            'NOT inv-nodes-6-8':5,
                            'deg-malign-1':5,
                            'NOT deg-malign-1':5,
                            'deg-malign-2':5,
                            'NOT deg-malign-2':5,
                            'deg-malign-3':5,
                            'NOT deg-malign-3':5
                           })

non_recurrence_rule_2 = Memory(0.5, 0.5,
                           {'menopause-lt40':5,
                            'NOT menopause-lt40':5,
                            'menopause-ge40':5,
                            'NOT menopause-ge40':5,
                            'menopause-premeno':5,
                            'NOT menopause-premeno':5,
                            'inv-nodes-0-2':5,
                            'NOT inv-nodes-0-2':5,
                            'inv-nodes-3-5':5,
                            'NOT inv-nodes-3-5':5,
                            'inv-nodes-6-8':5,
                            'NOT inv-nodes-6-8':5,
                            'deg-malign-1':5,
                            'NOT deg-malign-1':5,
                            'deg-malign-2':5,
                            'NOT deg-malign-2':5,
                            'deg-malign-3':5,
                            'NOT deg-malign-3':5
                           })

In [62]:
# Feedback for recurrence rule
for i in range(500):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 0:
        type_i_feedback(patients_recurrence[observation_id], recurrence_rule_2)
    else:
        type_ii_feedback(patients_non_recurrence[observation_id], recurrence_rule_2)

# Feedback for non recurrence rule
for i in range(500):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 0:
        type_i_feedback(patients_non_recurrence[observation_id], non_recurrence_rule_2)
    else:
        type_ii_feedback(patients_recurrence[observation_id], non_recurrence_rule_2)


In [63]:
print(classify(patients_recurrence[0], [recurrence_rule_2], [non_recurrence_rule_2]))
print(classify(patients_recurrence[1], [recurrence_rule_2], [non_recurrence_rule_2]))
print(classify(patients_recurrence[2], [recurrence_rule_2], [non_recurrence_rule_2]))

print(classify(patients_non_recurrence[0], [recurrence_rule_2], [non_recurrence_rule_2]))
print(classify(patients_non_recurrence[1], [recurrence_rule_2], [non_recurrence_rule_2]))
print(classify(patients_non_recurrence[2], [recurrence_rule_2], [non_recurrence_rule_2]))

print("\nIF " + " AND ".join(recurrence_rule_2.get_condition()) + " THEN Recurrence \n")

print("IF " + " AND ".join(non_recurrence_rule_2.get_condition()) + " THEN Non-Recurrence")

Recurrence
Recurrence
Recurrence
Recurrence
Non Recurrence
Non Recurrence

IF NOT menopause-lt40 AND NOT inv-nodes-3-5 AND NOT deg-malign-1 AND NOT deg-malign-2 AND deg-malign-3 THEN Recurrence 

IF NOT menopause-lt40 AND inv-nodes-0-2 AND NOT inv-nodes-3-5 AND NOT inv-nodes-6-8 AND NOT deg-malign-3 THEN Non-Recurrence


## Step 8
Learning rules with forget value 0.2 and memorize value 0.8

In [64]:
recurrence_rule_3 = Memory(0.2, 0.8,
                           {'menopause-lt40':5,
                            'NOT menopause-lt40':5,
                            'menopause-ge40':5,
                            'NOT menopause-ge40':5,
                            'menopause-premeno':5,
                            'NOT menopause-premeno':5,
                            'inv-nodes-0-2':5,
                            'NOT inv-nodes-0-2':5,
                            'inv-nodes-3-5':5,
                            'NOT inv-nodes-3-5':5,
                            'inv-nodes-6-8':5,
                            'NOT inv-nodes-6-8':5,
                            'deg-malign-1':5,
                            'NOT deg-malign-1':5,
                            'deg-malign-2':5,
                            'NOT deg-malign-2':5,
                            'deg-malign-3':5,
                            'NOT deg-malign-3':5
                           })

non_recurrence_rule_3 = Memory(0.2, 0.8,
                           {'menopause-lt40':5,
                            'NOT menopause-lt40':5,
                            'menopause-ge40':5,
                            'NOT menopause-ge40':5,
                            'menopause-premeno':5,
                            'NOT menopause-premeno':5,
                            'inv-nodes-0-2':5,
                            'NOT inv-nodes-0-2':5,
                            'inv-nodes-3-5':5,
                            'NOT inv-nodes-3-5':5,
                            'inv-nodes-6-8':5,
                            'NOT inv-nodes-6-8':5,
                            'deg-malign-1':5,
                            'NOT deg-malign-1':5,
                            'deg-malign-2':5,
                            'NOT deg-malign-2':5,
                            'deg-malign-3':5,
                            'NOT deg-malign-3':5
                           })

In [65]:
# Feedback for recurrence rule
for i in range(500):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 0:
        type_i_feedback(patients_recurrence[observation_id], recurrence_rule_3)
    else:
        type_ii_feedback(patients_non_recurrence[observation_id], recurrence_rule_3)

# Feedback for non recurrence rule
for i in range(500):
    observation_id = choice([0,1,2])
    patient = choice([0,1])
    if patient == 0:
        type_i_feedback(patients_non_recurrence[observation_id], non_recurrence_rule_3)
    else:
        type_ii_feedback(patients_recurrence[observation_id], non_recurrence_rule_3)

In [66]:
print(classify(patients_recurrence[0], [recurrence_rule_3], [non_recurrence_rule_3]))
print(classify(patients_recurrence[1], [recurrence_rule_3], [non_recurrence_rule_3]))
print(classify(patients_recurrence[2], [recurrence_rule_3], [non_recurrence_rule_3]))

print(classify(patients_non_recurrence[0], [recurrence_rule_3], [non_recurrence_rule_3]))
print(classify(patients_non_recurrence[1], [recurrence_rule_3], [non_recurrence_rule_3]))
print(classify(patients_non_recurrence[2], [recurrence_rule_3], [non_recurrence_rule_3]))

print("\nIF " + " AND ".join(recurrence_rule_3.get_condition()) + " THEN Recurrence \n")

print("IF " + " AND ".join(non_recurrence_rule_3.get_condition()) + " THEN Non-Recurrence")

Recurrence
Recurrence
Recurrence
Recurrence
Recurrence
Non Recurrence

IF NOT menopause-lt40 AND NOT menopause-ge40 AND menopause-premeno AND inv-nodes-0-2 AND NOT inv-nodes-6-8 AND NOT deg-malign-1 AND NOT deg-malign-2 AND deg-malign-3 THEN Recurrence 

IF NOT menopause-lt40 AND NOT menopause-ge40 AND menopause-premeno AND inv-nodes-0-2 AND NOT inv-nodes-3-5 AND NOT inv-nodes-6-8 AND deg-malign-1 AND NOT deg-malign-2 AND NOT deg-malign-3 THEN Non-Recurrence


## All rules combined

In [67]:
recurrence_rules = [recurrence_rule_1, recurrence_rule_2, recurrence_rule_3]
non_recurrence_rules = [non_recurrence_rule_1, non_recurrence_rule_2, non_recurrence_rule_3]

print(classify(patients_recurrence[0], recurrence_rules, non_recurrence_rules))
print(classify(patients_recurrence[1], recurrence_rules, non_recurrence_rules))
print(classify(patients_recurrence[2], recurrence_rules, non_recurrence_rules))

print(classify(patients_non_recurrence[0], recurrence_rules, non_recurrence_rules))
print(classify(patients_non_recurrence[1], recurrence_rules, non_recurrence_rules))
print(classify(patients_non_recurrence[2], recurrence_rules, non_recurrence_rules))

print("\nIF " + " AND ".join(recurrence_rule_1.get_condition()) + " THEN Recurrence")
print("IF " + " AND ".join(recurrence_rule_2.get_condition()) + " THEN Recurrence")
print("IF " + " AND ".join(recurrence_rule_3.get_condition()) + " THEN Recurrence \n")

print("IF " + " AND ".join(non_recurrence_rule_1.get_condition()) + " THEN Non-Recurrence")
print("IF " + " AND ".join(non_recurrence_rule_2.get_condition()) + " THEN Non-Recurrence")
print("IF " + " AND ".join(non_recurrence_rule_3.get_condition()) + " THEN Non-Recurrence")

Recurrence
Recurrence
Recurrence
Recurrence
Non Recurrence
Non Recurrence

IF NOT menopause-lt40 AND NOT deg-malign-1 AND NOT deg-malign-2 AND deg-malign-3 THEN Recurrence
IF NOT menopause-lt40 AND NOT inv-nodes-3-5 AND NOT deg-malign-1 AND NOT deg-malign-2 AND deg-malign-3 THEN Recurrence
IF NOT menopause-lt40 AND NOT menopause-ge40 AND menopause-premeno AND inv-nodes-0-2 AND NOT inv-nodes-6-8 AND NOT deg-malign-1 AND NOT deg-malign-2 AND deg-malign-3 THEN Recurrence 

IF inv-nodes-0-2 AND NOT deg-malign-3 THEN Non-Recurrence
IF NOT menopause-lt40 AND inv-nodes-0-2 AND NOT inv-nodes-3-5 AND NOT inv-nodes-6-8 AND NOT deg-malign-3 THEN Non-Recurrence
IF NOT menopause-lt40 AND NOT menopause-ge40 AND menopause-premeno AND inv-nodes-0-2 AND NOT inv-nodes-3-5 AND NOT inv-nodes-6-8 AND deg-malign-1 AND NOT deg-malign-2 AND NOT deg-malign-3 THEN Non-Recurrence
